# Assignment 8, GR10NR8

This program will illustrate a model that solves Fick’s 2nd law of diffusion in one dimension. The model will be used to investigate decarburization during heat treatment of a steel plate.

The change in concentration over time is described by Fick's 2nd law:

$$
\begin{align*}
\frac{\partial c}{\partial t} = D \frac{\partial^2 c}{\partial x^2}\tag{1}
\label{eq1}
\end{align*}
$$

where c is concentration, t is time, x is postition in the sample and $D$ is the diffusion constant given by:

$$
\begin{align*}
D = D_0 exp\left(-\frac{Q}{RT}\right)\tag{2}
\label{eq2}
\end{align*}
$$

where
<ul>
    <li> D<sub>0</sub> is a material constant and equal to 0.234*10<sup>-4</sup> </li>
    <li> Q is the activation energy and equal to 1.478*10<sup>5</sup> J/mol </li>
    <li> R is the gas constant </li>
    <li> T is the temperature </sup>


In [207]:
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display
from IPython.html.widgets import *
from IPython.display import clear_output
import pandas as pd

newparams = {'figure.figsize': (15, 9), 'axes.grid': False,
             'lines.markersize': 10, 'lines.linewidth': 2,
             'font.size': 15, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral', 'figure.dpi': 200}
plt.rcParams.update(newparams)

In [215]:
#Constants
D0 = 0.234*10**(-4) #m**2/s
Q = 1.478*10**5 #J/mol
R = 8.314 #J/mol*K
L = 1*10**(-3) #m

### Numerical solution

The numerical solution is calculated by first finding the derivative with respect to position, $\frac{\Delta C}{\Delta x}$ as well as the second derivative $\frac{\Delta^2 C}{\Delta x^2}$. Afterwards $\frac{\Delta C}{\Delta t}$ and $\Delta C$ is calculated by Fick's 2nd law.

In [208]:
x = np.linspace(0, L/2, 20)
dx = (x[1]-x[0])
c0 = np.zeros(19)+1
c0 = np.insert(c0, 0, 0)

In [209]:
def ddcddx(c): #finding the first and secondary derivative of concentration with respect to position.
    dcdx = np.zeros(20)
    for i in range(1,len(c)):
        dcdx[-i] = ((c[i]-c[i-1])/(dx))
    dcdx = np.flip(dcdx) #in order to get all the values.
    dcdx2 = []
    for i in range(0, len(dcdx)-1):
        dcdx2.append((dcdx[i+1]-dcdx[i])/dx)
    return dcdx2

In [210]:
#calculating the first derivative of concentration with respect to time using Fick's 2nd law.
def dcdt(T, c): 
    D = D0*np.exp(-Q/(R*T))
    dcdx2 = ddcddx(c)
    #Values for dC/dt:
    dcdt=np.empty(0)
    for i in range(len(dcdx2)):
        a = D*dcdx2[i]
        dcdt = np.append(dcdt, a)
    dcdt = np.insert(dcdt, 0, 0)
    return dcdt  

        
#New values for the consentration using Fick's 2nd law.
def cnew(T, dt,c):
    dcdt1 = dcdt(T,c)
    c_ny = c+dt*dcdt1
    return c_ny

In [211]:
#Calculating c(x) for all t in [0, 60]min
def newCprofile(T, dt): 
    t = 0
    liste = np.empty(0)
    c = c0
    while t <= 3600:
        c = cnew(T, dt, c)
        
        if t % 300 == 0:
            liste = np.append(liste, c)
        t+=dt
    liste = np.split(liste, 13)
    return liste

### Analytical solution

The analytical concentrations were found by using the formula based on the Fourier series.

$$ C(x,t) = \frac{4C_{0}}{\pi}\sum_{i=0}^{k=\infty}\frac{1}{2i+1}sin(\frac{(2i+1)\pi(x)}{L}exp(-\frac{t}{\tau_{i}}) $$

where

$$ \tau_{i} = \frac{L^{2}}{(2i+1)^{2}\pi^{2}D} $$

where L is the thickness of the sample and $C_0$ is the initial concentration and equal to 1wt% in our case.

In [212]:
def analytical(T):
    t = np.arange(300, 3601, 300)
    D = D0*np.exp(-Q/(R*T))
   
    c_0 = 1
    ca = np.zeros((len(t), len(x)))
    for e in range(len(t)):
        cx=np.zeros(len(c0))
        for r in range(len(x)):
            s = 0
            for i in range(25):
                tau = L**2/((2*i+1)**2*np.pi**2*D)
                s += (4*c_0)/np.pi*(1/(2*i+1))*np.sin(((2*i+1)*np.pi*x[r])/L)*np.exp(-t[e]/tau)
            cx[r] = s
        ca[e] = cx        
    return ca,t

### Mean concentration

The mean concentration is given by the following formula,

$$ c = \frac{1}{20}\sum_{n=1}^{20}(c_{n}+c_{n+1})/2 $$

In [ ]:
def meanC(c):
    cmean = 0
    for i in range(len(c)-1):
        cmean += (1/20)*(c[i]+c[i+1])/2
    return cmean

### Plot

In [221]:
#Plotting the graphs and making the table.
def plot(T, dt):
    D = D0*np.exp(-Q/(R*T))
    if (D*dt)/dx**2 < 0.5:
        colors = ['black','grey', 'brown', 'red', 'peachpuff', 'orange', 'chartreuse', 'limegreen', 'turquoise', 'cyan', 'blue', 'hotpink', 'crimson']
        cn = newCprofile(T, dt)
        ca, t = analytical(T)
        plt.plot(x, c0, label='t =0 min', color='k')
        col = 0
        for i in range(len(t)):
            plt.plot(x, ca[i], label="t ="+ str(t[i]/60) + " min", color=colors[col+1])
            plt.plot(x, cn[i], linestyle='--', color=colors[col])
            col += 1
        plt.legend()
        plt.title('Numerical vs. Analytical concentration for diffusion')
        plt.xlabel('Distance x, [m]')
        plt.ylabel('Concentration of carbon, [%]')
        plt.grid()
        cam= np.empty(0)
        cnm= np.empty(0)
        for i in range(len(t)):
            cam = np.append(cam, meanC(ca[i]))
            cnm= np.append(cam, meanC(cn[i]))
        cam = np.insert(cam,0, meanC(c0)) 
        t = np.insert(t, 0, 0)
        df = pd.DataFrame({
        'Time, [min]': t/60,
        'Mean analytical concentration, [%]': np.round(cam, 3),
        'Mean numerical concentration, [%]': np.round(cnm, 3),
        })
        display(df)
    else:
        print("Error! The values doesn't give a numerical stable solution, try with a lower dt or T") 

### Interactive menu

In [222]:
#Buttons
T_suggested = widgets.ToggleButtons(
    options=[('900 °C',1173),('1000 °C',1273),('1050 °C',1323), ('1150 °C',1423)],
    value=1273,
    description = 'T',
    disabled=False,
) 

dt_suggested =  widgets.ToggleButtons(
    options=[('1s',1),('3s',3), ('5s',5),('10s',10)],
    value=5,
    description = 'dt',
    disabled=False,
) 


def program(): #Interactive buttons and sliders for T and dt 
    clear_output(wait=True)
    interact(plot, T=T_suggested, dt=dt_suggested)

interact(program)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

<function __main__.program()>

The dashed lines are the numerical concentrations while the solid lines are the analytical concentrations.